In [1]:
import os
import time
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver
# database python file
import mysql.connector 
from mysql.connector import Error
from sqlalchemy import create_engine


In [2]:
# database functions
## Create connection
def get_connection():
    return mysql.connector.connect(host='127.0.0.1',
                                   user='root',
                                   password='',
                                   charset="utf8mb4")

## create a Assesment database
def create_db():
    try:
        connection = get_connection()
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute("CREATE DATABASE IF NOT EXISTS assesment;")
            print("Database is created...")
            connection.commit()
    except Exception as e:
        print("Error while connecting to MySQL", e)


## create tables 
def create_table():
    try:
        connection = get_connection()
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute("USE assesment;")
            cursor.execute(
                """CREATE TABLE IF NOT EXISTS urls( video_url VARCHAR(255),keyword VARCHAR(255)) ;""") 
            # comment table
            #'primary_keyword','secondary_keyword','video_url','content','comment','authors_name','comment_date'
            cursor.execute( """CREATE TABLE IF NOT EXISTS comments(keyword VARCHAR(100),video_url VARCHAR(255), content TEXT, comment TEXT, authors_name VARCHAR(255) ,comment_date VARCHAR(255));""")
            print("Table is created...")
            connection.commit()
    except Exception as e:
        print("Error while connecting to MySQL", e)

## execute any queries on database
def execute_query(query):
    try:
        connection = get_connection()
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute("USE assesment;")
            cursor.execute(query)
            connection.commit()
            print("Query finished")
    except Exception as e:
            print("Query is failed:" + query)
    
## get query result as a dataframe
def get_data(query):
    try:
        connection = get_connection()
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute("USE assesment;")
            cursor.execute(query)
            cols = list(map(lambda x: x[0], cursor.description))
            #result_list=cursor.fetchall()
    except Exception as e:
            print("Query is failed")
    df=pd.DataFrame(cursor.fetchall(),columns=cols)
    return df

## insert data to table from dataframe
def insertData(df,tablename):
    try:
        conn = create_engine("mysql://root@localhost/assesment")
        df.to_sql(tablename,con=conn,if_exists='append',index=False)
        print("Inserting complete...")
    except Exception as e:
        print("Insertin failed :",e)


In [3]:
#utilization for selenium browser
#all folder and files
dir_path = os.path.dirname(os.path.realpath("assesment_cct"))

def new_browser():

    #Global Variables
    PATH = "chromedriver.exe"
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("start-maximized")

    # Chrome is controlled by automated test software
    chromeOptions.add_experimental_option("excludeSwitches", ["enable-automation"])
    chromeOptions.add_experimental_option('useAutomationExtension', False)

    # avoiding detection
    chromeOptions.add_argument('--disable-blink-features=AutomationControlled')

    # Default User Profile
    chromeOptions.add_argument("--profile-directory=Default")
    chromeOptions.add_argument("--user-data-dir=C:/ChromeProfiles/User Data")
    #chromeOptions.add_argument("--user-data-dir=C:/Users/Admin/AppData/Local/Google/Chrome/User Data")

    #Setting up webdriver
    #prefs = {"download.default_directory" : dir_path + "/files/}
    prefs = {"download.default_directory" : dir_path}
    chromeOptions.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome(executable_path=PATH, options=chromeOptions)
    driver.set_script_timeout(1000)
    return driver

In [7]:
# scraping video url about Ireland housing crisis
## HTML elements' class names for scraping
LoadMoreVideos = 'e17vxm6m1'
videoBlockAll='eegew6e0'
videoUrl='e1cg0wnj1'

#scroll page 10 times
def scrollPage(driver):
    print("scroll....")
    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(10):
        # Scroll down to drivertom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        try:
            driver.find_element(By.CLASS_NAME,LoadMoreVideos).click()
            # Wait to load page
            time.sleep(1)
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        except Exception as err:
            print("There is no video...")
            return err

## get video urls 
def getVideoUrl(driver,srcKeyword):
    print("Scraping urls...")
    links=[]
    #all videos element
    blk=driver.find_element(By.CLASS_NAME,videoBlockAll)
    videos=blk.find_elements(By.TAG_NAME,'a')
    links=[]
    for a in videos:
        if str(a.get_attribute('href')).startswith("https://www.tiktok.com/@") and a.get_attribute('href'): 
            if "/video/" in str(a.get_attribute('href')):
                links.append(a.get_attribute('href'))
    #print(len(links))
    df = pd.DataFrame()  
    df["video_url"]=links
    df["keyword"]=srcKeyword
    return df

def searchKeywords(driver,srcKeyword):
    print("searching keywords :" + srcKeyword)
    search_url =  str.replace(srcKeyword,' ','%20')
    driver.get("https://www.tiktok.com/search?q=" + search_url)
    time.sleep(5)
    scrollPage(driver)

In [5]:
## create db and tables
create_db()
create_table()

Database is created...
Table is created...


In [8]:
# save urls at the db
driver=new_browser()
srcKeyword="housing crisis Ireland"
searchKeywords(driver,srcKeyword)
df=getVideoUrl(driver,srcKeyword)
insertData(df,'urls')

/var/folders/py/4cct40495x36bqlv8mh9q3m00000gn/T/ipykernel_10060/2568263973.py:28: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=PATH, options=chromeOptions)


WebDriverException: Message: unknown error: Chrome failed to start: exited normally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /Applications/Google Chrome.app/Contents/MacOS/Google Chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
0   chromedriver                        0x0000000107bc1598 chromedriver + 4973976
1   chromedriver                        0x0000000107bb8913 chromedriver + 4938003
2   chromedriver                        0x0000000107775cd6 chromedriver + 470230
3   chromedriver                        0x00000001077a3f0d chromedriver + 659213
4   chromedriver                        0x000000010779fb79 chromedriver + 641913
5   chromedriver                        0x00000001077e72a6 chromedriver + 934566
6   chromedriver                        0x00000001077e675f chromedriver + 931679
7   chromedriver                        0x00000001077dd793 chromedriver + 894867
8   chromedrive